In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import io
import zipfile
import xml.etree.ElementTree as et 
import json
import requests

In [10]:
path = "/Users/kimhansaem/workspace/11.DataAnalysis/05.신용평가모형/data/"

In [11]:
df_test = pd.read_csv(path + 'test_psi.csv')
df_test.head()

,kind,id,engine_type,grade
0,201512,1,0,5
1,201512,2,0,5
2,201512,3,1,4
3,201512,4,1,4
4,201512,5,0,7


In [12]:
df_test.shape

(64, 4)

#### 개발시점 등급별 건수 산출

In [13]:
grd_dist_dev = pd.DataFrame(df_test[df_test["kind"] == 201512].groupby("grade")
["id"].count().reset_index())

#### 검증시점 등급별 건수 산출

In [14]:
grd_dist_prf = pd.DataFrame(df_test[df_test["kind"] == 201612].groupby("grade")
["id"].count().reset_index())

#### 개발시점과 검증 시점의 등급별 구성비 산출

In [15]:
grd_dist_dev["tot_pct"] = grd_dist_dev["id"]/grd_dist_dev["id"].sum()
grd_dist_prf["tot_pct"] = grd_dist_prf["id"]/grd_dist_prf["id"].sum()


In [16]:
psi_table = pd.merge(grd_dist_dev,grd_dist_prf, how = "left", on="grade")

In [21]:
for i in range(psi_table["grade"].size):
    psi_table.loc[i,"psi"]=(psi_table.loc[i,"tot_pct_x"]-psi_table.loc[i,"tot_pct_y"])*log(psi_table.loc[i,"tot_pct_x"]/psi_table.loc[i,"tot_pct_y"])

round(psi_table.psi.sum(),8)

0.02166085

## 다른 예시

In [ ]:
import numpy as np

def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    '''Calculate the PSI (population stability index) across all variables

    Args:
       expected: numpy matrix of original values
       actual: numpy matrix of new values, same size as expected
       buckettype: type of strategy for creating buckets, bins splits into even splits, quantiles splits into quantile buckets
       buckets: number of quantiles to use in bucketing variables
       axis: axis by which variables are defined, 0 for vertical, 1 for horizontal

    Returns:
       psi_values: ndarray of psi values for each variable

    Author:
       Matthew Burke
       github.com/mwburke
       worksofchart.com
    '''

    def psi(expected_array, actual_array, buckets):
        '''Calculate the PSI for a single variable

        Args:
           expected_array: numpy array of original values
           actual_array: numpy array of new values, same size as expected
           buckets: number of percentile ranges to bucket the values into

        Returns:
           psi_value: calculated PSI value
        '''

        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input


        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            '''Calculate the actual PSI value from comparing the values.
               Update the actual value to a very small number if equal to zero
            '''
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)